# Optic Nerve Left sensitivity results

This notebook contains code to reproduce what is reported in Table 2 of the paper.

In [2]:
# Setup library paths
import os
import numpy as np
import SimpleITK as sitk

def volumetric_dice(img, mask):
    img[img > 1] = 1
    img[img < 1] = 0
    mask[mask > 1] = 1
    mask[mask < 1] = 0
    return np.sum(mask[img == mask]) * 2.0 / (np.sum(img)+np.sum(mask))

root_path = "/Users/amithkamath/repo/deepdosesens"
base_gt_path = os.path.join(root_path, "data", "processed-ONL")
base_pred_path = os.path.join(root_path, "data", "output-ONL", "output-ONL-6", "Prediction")

In [3]:
mean_ONL_dose = []

for pred_idx in range(0, 10):
    pred_dose = sitk.ReadImage(os.path.join(base_pred_path, "DLDP_" + str(pred_idx).zfill(3), "Dose.nii.gz"))
    pred_dose = sitk.GetArrayFromImage(pred_dose)
    
    pred_mask = sitk.ReadImage(os.path.join(base_gt_path, "DLDP_" + str(pred_idx).zfill(3), "OpticNerve_L.nii.gz"))
    pred_mask = sitk.GetArrayFromImage(pred_mask)

    mean_pred = np.mean(pred_dose[pred_mask > 0])
    mean_ONL_dose.append(mean_pred)

print(mean_ONL_dose)

[35.54512, 35.767376, 32.374115, 34.51261, 34.062115, 30.108437, 35.963802, 41.170876, 33.057617, 36.064495]


Compute the difference between the reference doses between the 0th and the rest of the variations of the ONL contour.

In [4]:
expert_conf_matrix = np.zeros((10))

first_idx = 0
first_dose = sitk.ReadImage(os.path.join(base_gt_path, "DLDP_" + str(first_idx).zfill(3), "Dose.nii.gz"))
first_dose = sitk.GetArrayFromImage(first_dose)

first_mask = sitk.ReadImage(os.path.join(base_gt_path, "DLDP_" + str(first_idx).zfill(3), "OpticNerve_L.nii.gz"))
first_mask = sitk.GetArrayFromImage(first_mask)

mean_first = np.mean(first_dose[first_mask > 0])

for second_idx in range(0, 10):
    second_dose = sitk.ReadImage(os.path.join(base_gt_path, "DLDP_" + str(second_idx).zfill(3), "Dose.nii.gz"))
    second_dose = sitk.GetArrayFromImage(second_dose)
    
    second_mask = sitk.ReadImage(os.path.join(base_gt_path, "DLDP_" + str(second_idx).zfill(3), "OpticNerve_L.nii.gz")) 
    second_mask = sitk.GetArrayFromImage(second_mask)

    mean_second = np.mean(second_dose[second_mask > 0])
    diff = np.abs(mean_first - mean_second)
    expert_conf_matrix[second_idx] = diff

expert_conf_matrix


array([0.        , 0.28360919, 2.08912787, 0.35797755, 3.02786854,
       4.81561849, 0.14585688, 7.59197052, 2.40275255, 0.43549413])

Compute the difference between the predicted doses between the 0th and the rest of the variations of the ONL contour.Also compute the dice scores between these contour variations.

In [5]:
model_conf_matrix = np.zeros((10))
dice_conf_matrix = np.zeros((10))

first_idx = 0
first_dose = sitk.ReadImage(os.path.join(base_pred_path, "DLDP_" + str(first_idx).zfill(3), "Dose.nii.gz"))
first_dose = sitk.GetArrayFromImage(first_dose)
first_mask = sitk.ReadImage(os.path.join(base_gt_path, "DLDP_" + str(first_idx).zfill(3), "OpticNerve_L.nii.gz"))
first_mask = sitk.GetArrayFromImage(first_mask)
mean_first = np.mean(first_dose[first_mask > 0])

for second_idx in range(0, 10):
    second_dose = sitk.ReadImage(os.path.join(base_pred_path, "DLDP_" + str(second_idx).zfill(3), "Dose.nii.gz"))
    second_dose = sitk.GetArrayFromImage(second_dose)

    second_mask = sitk.ReadImage(os.path.join(base_gt_path, "DLDP_" + str(second_idx).zfill(3), "OpticNerve_L.nii.gz")) 
    second_mask = sitk.GetArrayFromImage(second_mask)

    mean_second = np.mean(second_dose[second_mask > 0])
    diff = np.abs(mean_first - mean_second)
    model_conf_matrix[second_idx] = diff
    dice_conf_matrix[second_idx] = volumetric_dice(first_mask, second_mask)


Create an index of the increasing order of differences between reference doses. Then, use this to plot the order of reference dose differences, predicted dose, and dice scores. This is what is reported in Table 2.

In [6]:
expert_order = np.argsort(np.abs(expert_conf_matrix))

expert_dose_diff = np.abs(expert_conf_matrix)[expert_order]
pred_dose_diff = np.abs(model_conf_matrix)[expert_order]
dice_diff = np.abs(dice_conf_matrix)[expert_order]

print("Dose differences among experts    : ", expert_dose_diff)
print("Dose differences among predictions: ", pred_dose_diff)
print("Dice score differences            : ", dice_diff)

Dose differences among experts    :  [0.         0.14585688 0.28360919 0.35797755 0.43549413 2.08912787
 2.40275255 3.02786854 4.81561849 7.59197052]
Dose differences among predictions:  [0.         0.4186821  0.22225571 1.03250885 0.51937485 3.17100525
 2.48750305 1.48300552 5.43668365 5.62575531]
Dice score differences            :  [1.         0.3255814  0.62790698 0.78333333 0.36363636 0.59016393
 0.50943396 0.19753086 0.61261261 0.22900763]


In [7]:
print("Mean absolute reference dose difference is: " + str(np.mean(expert_dose_diff)))

Mean absolute reference dose difference is: 2.1150275713423934


In [8]:
print("Mean absolute predicted dose difference is: " + str(np.mean(pred_dose_diff)))

Mean absolute predicted dose difference is: 2.0396774291992186


In [9]:
print("Mean absolute dice score is: " + str(np.mean(dice_diff)))

Mean absolute dice score is: 0.523920707615103


Compute the correlation coefficient between dice score and reference dose differences.

In [10]:
np.corrcoef(expert_dose_diff, dice_diff)

array([[ 1.        , -0.47148969],
       [-0.47148969,  1.        ]])

Compute the correlation coefficient between predicted dose differences and reference dose differences.

In [11]:
np.corrcoef(expert_dose_diff, pred_dose_diff)

array([[1.        , 0.92649897],
       [0.92649897, 1.        ]])